<a href="https://colab.research.google.com/github/its3alih/Thesis/blob/main/FewShots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##FIRST

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# Load the Excel file
df = pd.read_excel("/content/IO.xlsx")  # Adjust path if needed

# Filter rows with valid tags
df = df[df["Word i entity tag"].isin(["B", "I", "O"])]

# Keep only necessary columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# Format row as few-shot prompt
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# Prepare few-shot examples
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# Load the generator
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# Test on next 100 tokens
y_true = []
y_pred = []
n_test = 100

for i in range(n_shots, n_shots + n_test):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input
    output = generator(full_prompt)[0]["generated_text"]

    # Extract prediction
    prediction = output.split("Entity Tag:")[-1].strip().split()[0]
    prediction = prediction if prediction in ["B", "I", "O"] else "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           I      0.348     0.400     0.372        20
           O      0.844     0.812     0.828        80

    accuracy                          0.730       100
   macro avg      0.596     0.606     0.600       100
weighted avg      0.745     0.730     0.737       100


📈 Evaluation Summary:
Accuracy:          0.7300
Macro Precision:   0.5960
Macro Recall:      0.6062
Macro F1 Score:    0.6001
Weighted F1 Score: 0.7368
Micro F1 Score:    0.7300


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the BI-tagged dataset ===
df = pd.read_excel("/content/BI.xlsx")  # Adjust path if needed

# Keep only B and I tags
df = df[df["Word i entity tag"].isin(["B", "I"])]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Construct few-shot prompt ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluation on next n_test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in ["B", "I"] else "I"  # default to "I"
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.761     0.745     0.753        47
           I      0.778     0.792     0.785        53

    accuracy                          0.770       100
   macro avg      0.769     0.769     0.769       100
weighted avg      0.770     0.770     0.770       100


📈 Evaluation Summary:
Accuracy:          0.7700
Macro Precision:   0.7693
Macro Recall:      0.7686
Macro F1 Score:    0.7689
Weighted F1 Score: 0.7698
Micro F1 Score:    0.7700


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the BIES-tagged dataset ===
df = pd.read_excel("/content/BIES.xlsx")  # Adjust this path to your BIES dataset

# Keep only valid BIES tags
valid_tags = ["B", "I", "E", "S"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Construct few-shot prompt ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on the next n_test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "I"  # fallback to "I"
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.839     0.684     0.754        38
           E      0.607     0.872     0.716        39
           I      0.429     0.214     0.286        14
           S      0.333     0.222     0.267         9

    accuracy                          0.650       100
   macro avg      0.552     0.498     0.505       100
weighted avg      0.645     0.650     0.630       100


📈 Evaluation Summary:
Accuracy:          0.6500
Macro Precision:   0.5519
Macro Recall:      0.4981
Macro F1 Score:    0.5054
Weighted F1 Score: 0.6295
Micro F1 Score:    0.6500


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IE-tagged dataset ===
df = pd.read_excel("/content/IE.xlsx")  # 🔁 Update this path if needed

# Keep only valid IE tags
valid_tags = ["I", "E"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot context ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the Falcon generator ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on test set ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "I"  # Fallback to I
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           E      0.717     0.688     0.702        48
           I      0.722     0.750     0.736        52

    accuracy                          0.720       100
   macro avg      0.720     0.719     0.719       100
weighted avg      0.720     0.720     0.720       100


📈 Evaluation Summary:
Accuracy:          0.7200
Macro Precision:   0.7198
Macro Recall:      0.7188
Macro F1 Score:    0.7190
Weighted F1 Score: 0.7197
Micro F1 Score:    0.7200


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOB-tagged dataset ===
df = pd.read_excel("/content/IOB.xlsx")  # 🔁 Update this path if needed

# Keep only valid IOB tags
valid_tags = ["B", "I", "O"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot context ===
n_shots = 5  # Change to 0 for pure zero-shot
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the text-generation pipeline ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # Fallback to O
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.175     0.875     0.292         8
           I      0.000     0.000     0.000        12
           O      0.867     0.650     0.743        80

    accuracy                          0.590       100
   macro avg      0.347     0.508     0.345       100
weighted avg      0.707     0.590     0.618       100


📈 Evaluation Summary:
Accuracy:          0.5900
Macro Precision:   0.3472
Macro Recall:      0.5083
Macro F1 Score:    0.3448
Weighted F1 Score: 0.6176
Micro F1 Score:    0.5900


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOBES-tagged dataset ===
df = pd.read_excel("/content/IOBES.xlsx")  # 🔁 Update this path if needed

# Valid IOBES tags
valid_tags = ["B", "I", "O", "E", "S"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot setup (set n_shots=0 for zero-shot) ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the generator ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # Fallback
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.115     0.600     0.194         5
           E      0.000     0.000     0.000         6
           I      0.000     0.000     0.000         6
           O      0.804     0.562     0.662        80
           S      0.000     0.000     0.000         3

    accuracy                          0.480       100
   macro avg      0.184     0.233     0.171       100
weighted avg      0.649     0.480     0.539       100


📈 Evaluation Summary:
Accuracy:          0.4800
Macro Precision:   0.1838
Macro Recall:      0.2325
Macro F1 Score:    0.1711
Weighted F1 Score: 0.5391
Micro F1 Score:    0.4800


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOE-tagged dataset ===
df = pd.read_excel("/content/IOE.xlsx")  # 🔁 Update this path if needed

# Valid IOE tags
valid_tags = ["I", "O", "E"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format a row as a prompt ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot setup (change n_shots = 0 for zero-shot) ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the text generation model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate the model ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # fallback to O
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           E      0.133     0.222     0.167         9
           I      0.100     0.182     0.129        11
           O      0.800     0.650     0.717        80

    accuracy                          0.560       100
   macro avg      0.344     0.351     0.338       100
weighted avg      0.663     0.560     0.603       100


📈 Evaluation Summary:
Accuracy:          0.5600
Macro Precision:   0.3444
Macro Recall:      0.3513
Macro F1 Score:    0.3376
Weighted F1 Score: 0.6030
Micro F1 Score:    0.5600


##SECOND

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# Load the Excel file
df = pd.read_excel("/content/IO2.xlsx")  # Adjust path if needed

# Filter rows with valid tags
df = df[df["Word i entity tag"].isin(["B", "I", "O"])]

# Keep only necessary columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# Format row as few-shot prompt
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# Prepare few-shot examples
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# Load the generator
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# Test on next 100 tokens
y_true = []
y_pred = []
n_test = 100

for i in range(n_shots, n_shots + n_test):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input
    output = generator(full_prompt)[0]["generated_text"]

    # Extract prediction
    prediction = output.split("Entity Tag:")[-1].strip().split()[0]
    prediction = prediction if prediction in ["B", "I", "O"] else "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           I      0.310     0.562     0.400        16
           O      0.901     0.762     0.826        84

    accuracy                          0.730       100
   macro avg      0.606     0.662     0.613       100
weighted avg      0.807     0.730     0.758       100


📈 Evaluation Summary:
Accuracy:          0.7300
Macro Precision:   0.6059
Macro Recall:      0.6622
Macro F1 Score:    0.6129
Weighted F1 Score: 0.7577
Micro F1 Score:    0.7300


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the BI-tagged dataset ===
df = pd.read_excel("/content/BI2.xlsx")  # Adjust path if needed

# Keep only B and I tags
df = df[df["Word i entity tag"].isin(["B", "I"])]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Construct few-shot prompt ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluation on next n_test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in ["B", "I"] else "I"  # default to "I"
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.730     0.509     0.600        53
           I      0.587     0.787     0.673        47

    accuracy                          0.640       100
   macro avg      0.659     0.648     0.636       100
weighted avg      0.663     0.640     0.634       100


📈 Evaluation Summary:
Accuracy:          0.6400
Macro Precision:   0.6585
Macro Recall:      0.6483
Macro F1 Score:    0.6364
Weighted F1 Score: 0.6342
Micro F1 Score:    0.6400


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the BIES-tagged dataset ===
df = pd.read_excel("/content/BIES2.xlsx")  # Adjust this path to your BIES dataset

# Keep only valid BIES tags
valid_tags = ["B", "I", "E", "S"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Construct few-shot prompt ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on the next n_test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "I"  # fallback to "I"
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.837     0.878     0.857        41
           E      0.837     0.878     0.857        41
           I      0.400     0.333     0.364         6
           S      0.778     0.583     0.667        12

    accuracy                          0.810       100
   macro avg      0.713     0.668     0.686       100
weighted avg      0.804     0.810     0.805       100


📈 Evaluation Summary:
Accuracy:          0.8100
Macro Precision:   0.7130
Macro Recall:      0.6682
Macro F1 Score:    0.6861
Weighted F1 Score: 0.8047
Micro F1 Score:    0.8100


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IE-tagged dataset ===
df = pd.read_excel("/content/IE2.xlsx")  # 🔁 Update this path if needed

# Keep only valid IE tags
valid_tags = ["I", "E"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot context ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the Falcon generator ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on test set ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "I"  # Fallback to I
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "I"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           E      0.737     0.528     0.615        53
           I      0.597     0.787     0.679        47

    accuracy                          0.650       100
   macro avg      0.667     0.658     0.647       100
weighted avg      0.671     0.650     0.645       100


📈 Evaluation Summary:
Accuracy:          0.6500
Macro Precision:   0.6668
Macro Recall:      0.6578
Macro F1 Score:    0.6471
Weighted F1 Score: 0.6452
Micro F1 Score:    0.6500


In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOB-tagged dataset ===
df = pd.read_excel("/content/IOB2.xlsx")  # 🔁 Update this path if needed

# Keep only valid IOB tags
valid_tags = ["B", "I", "O"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot context ===
n_shots = 5  # Change to 0 for pure zero-shot
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the text-generation pipeline ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate on test examples ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # Fallback to O
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.176     0.750     0.286         8
           I      0.000     0.000     0.000         8
           O      0.864     0.679     0.760        84

    accuracy                          0.630       100
   macro avg      0.347     0.476     0.349       100
weighted avg      0.740     0.630     0.661       100


📈 Evaluation Summary:
Accuracy:          0.6300
Macro Precision:   0.3467
Macro Recall:      0.4762
Macro F1 Score:    0.3486
Weighted F1 Score: 0.6613
Micro F1 Score:    0.6300


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOBES-tagged dataset ===
df = pd.read_excel("/content/IOBES2.xlsx")  # 🔁 Update this path if needed

# Valid IOBES tags
valid_tags = ["B", "I", "O", "E", "S"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format rows as few-shot examples ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot setup (set n_shots=0 for zero-shot) ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the generator ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # Fallback
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Results ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           B      0.115     0.600     0.194         5
           E      0.000     0.000     0.000         6
           I      0.000     0.000     0.000         2
           O      0.810     0.560     0.662        84
           S      0.067     0.333     0.111         3

    accuracy                          0.510       100
   macro avg      0.198     0.299     0.193       100
weighted avg      0.688     0.510     0.569       100


📈 Evaluation Summary:
Accuracy:          0.5100
Macro Precision:   0.1985
Macro Recall:      0.2986
Macro F1 Score:    0.1933
Weighted F1 Score: 0.5691
Micro F1 Score:    0.5100


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
# Install required packages
!pip install pandas transformers scikit-learn openpyxl --quiet

import pandas as pd
from transformers import pipeline
from sklearn.metrics import (
    classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)

# === Load and prepare the IOE-tagged dataset ===
df = pd.read_excel("/content/IOE2.xlsx")  # 🔁 Update this path if needed

# Valid IOE tags
valid_tags = ["I", "O", "E"]
df = df[df["Word i entity tag"].isin(valid_tags)]

# Keep only the relevant columns
columns = [
    "Word i", "Word i entity tag", "Word i POS", "Stopword",
    "Word i Gazetteers", "Word i Lexical marker", "Word i definiteness"
]
df = df[columns].reset_index(drop=True)

# === Format a row as a prompt ===
def row_to_prompt(row):
    return (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag: {row['Word i entity tag']}\n"
    )

# === Few-shot setup (change n_shots = 0 for zero-shot) ===
n_shots = 5
few_shot_prompt = ""
for i in range(n_shots):
    few_shot_prompt += row_to_prompt(df.iloc[i]) + "\n"

# === Load the text generation model ===
generator = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=10)

# === Evaluate the model ===
n_test = 100
y_true, y_pred = [], []

for i in range(n_shots, min(n_shots + n_test, len(df))):
    row = df.iloc[i]

    test_input = (
        f"Word: {row['Word i']}\n"
        f"POS: {row['Word i POS']}\n"
        f"Stopword: {row['Stopword']}\n"
        f"Gazetteer: {row['Word i Gazetteers']}\n"
        f"Lexical marker: {row['Word i Lexical marker']}\n"
        f"Definiteness: {row['Word i definiteness']}\n"
        f"Entity Tag:"
    )

    full_prompt = few_shot_prompt + test_input

    try:
        output = generator(full_prompt)[0]["generated_text"]
        prediction = output.split("Entity Tag:")[-1].strip().split()[0]
        prediction = prediction if prediction in valid_tags else "O"  # fallback to O
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")
        prediction = "O"

    y_true.append(row["Word i entity tag"])
    y_pred.append(prediction)

# === Evaluation Metrics ===
print("\n📊 Classification Report:\n")
print(classification_report(y_true, y_pred, digits=3))

print("\n📈 Evaluation Summary:")
print(f"Accuracy:          {accuracy_score(y_true, y_pred):.4f}")
print(f"Macro Precision:   {precision_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro Recall:      {recall_score(y_true, y_pred, average='macro'):.4f}")
print(f"Macro F1 Score:    {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"Weighted F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:    {f1_score(y_true, y_pred, average='micro'):.4f}")


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 


📊 Classification Report:

              precision    recall  f1-score   support

           E      0.231     0.333     0.273         9
           I      0.161     0.714     0.263         7
           O      0.911     0.607     0.729        84

    accuracy                          0.590       100
   macro avg      0.434     0.552     0.421       100
weighted avg      0.797     0.590     0.655       100


📈 Evaluation Summary:
Accuracy:          0.5900
Macro Precision:   0.4343
Macro Recall:      0.5516
Macro F1 Score:    0.4215
Weighted F1 Score: 0.6550
Micro F1 Score:    0.5900
